In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math as math
import itertools as itertools
from operator import add, neg
import nashpy as nash
from matplotlib.backends.backend_pdf import PdfPages
import tqdm as tqdm    
from sympy import*

# Simplex algorithm (Pivot method)

In [3]:
def smallest_ratio(A):
    R=[]
    for j in range(A.shape[1]-1):
        for k in range(A.shape[0]-1):
            ratio=A[k,3]+5000
            if A[k,j]!=0:
                ratio=A[k,3]/A[k,j]
            R+=[expand(ratio)]
    return R

def vistaprinter(mat, COL):
    print("\n Value")
    print(mat[-1,-1])
    cols=mat.shape[0]-1
    ratios=smallest_ratio(mat)
    if COL==0:
        return(np.asarray(smallest_ratio(mat)[0:cols]))
    elif COL==1:
        return(np.asarray(smallest_ratio(mat)[cols:2*cols]))
    else:
        return(np.asarray(smallest_ratio(mat)[2*cols:3*cols]))
    
def pivot(row,col,mat):
    B=np.copy(mat)
    B[row,col]=1/mat[row,col]
    
    rowdim=int(mat.shape[0])
    coldim=int(mat.shape[1]) #
    
    for k in range(rowdim):
        for j in range(0,coldim):
            if k!=row and j!=col:
                p=mat[row,col]
                r=mat[row,j]
                c=mat[k,col]
                q=mat[k,j]

                smallmat=np.array([[p,r],[c,q]])
                transmat=pivot_transform(p,r,c,q)
                B[row,j]=transmat[0,1]
                B[k,col]=transmat[1,0]
                B[k,j]=transmat[1,1]
                
    return(B)

In [5]:
def pivot_transform(p,r,c,q):
    return np.array([[1/p,r/p],[-c/p,q-r*c/p]])

class Pivot_method():
    def __init__(self,A):
        
        hsmat=np.zeros_like(A)
        linspC=np.linspace(1,A.shape[0],A.shape[0])
        hsmat=np.insert(hsmat,A.shape[1],linspC,axis=1)
        linspR=np.hstack((np.linspace(1,A.shape[1],A.shape[1])*-1,np.asarray([0])))
        self.hsmat=np.insert(hsmat,A.shape[0],linspR,axis=0)
        
        
        A=np.insert(A,A.shape[1],np.repeat(1,A.shape[0]),axis=1)
        A=np.insert(A,A.shape[0],np.array([-1 for k in range(A.shape[1]-1)]+[0]),axis=0)
        self.A=A
        self.ROW=self.A.shape[0]
        self.COL=self.A.shape[1]
        self.permrow=[]
        self.permcol=[]


    def find_pivot(self):
        # Select Column
        i=0
        col_piv=-1
        while i< self.COL-1:
            if self.A[self.ROW-1,i]<0:
                col_piv=i
                break
            i+=1 
        if col_piv==-1:
            return[-100,-100]

        R=[]
        for k in range(self.ROW-1): #éviter colonner value
            ratio=5e10
            if self.A[k,col_piv]!=0:
                ratio=self.A[k,self.COL-1]/self.A[k,col_piv]
            R+=[ratio]

        arr=np.array(R)
        sort_idx=np.argsort(arr)
        j=0
        row_piv=sort_idx[0]
        while self.A[int(row_piv), col_piv] <=0:
            row_piv=sort_idx[j]
            j+=1
            if j>=self.ROW:
                return([-100,-100])
            
        self.permrow+=[row_piv]
        self.permcol+=[col_piv]
        
        self.hsmat[-1,col_piv],self.hsmat[row_piv,-1]=self.hsmat[row_piv,-1],self.hsmat[-1,col_piv]
        
        
        return row_piv, col_piv 

    def pivot(self, row, col):
        B=np.copy(self.A)
        B[row,col]=1/self.A[row,col]
        rowdim=int(self.ROW)
        coldim=int(self.COL)
        for k in range(rowdim):
            for j in range(0,coldim):
                if k!=row and j!=col:
                    p=self.A[row,col]
                    r=self.A[row,j]
                    c=self.A[k,col]
                    q=self.A[k,j]
                    smallmat=np.array([[p,r],[c,q]])
                    transmat=pivot_transform(p,r,c,q)
                    B[row,j]=transmat[0,1]
                    B[k,col]=transmat[1,0]
                    B[k,j]=transmat[1,1]     
        return(B)

    def printer(self, hider):
        value=1/self.A[self.ROW-1, self.COL-1]
        
        if hider:
            H=[]
            for h in range(1,self.COL):
                permuth=self.hsmat[:,-1]*-1
                if h in permuth:
                    H+=[self.A[int(list(permuth).index(h)),-1]*value]
                else:
                    H+=[0.0]

            S=[]
            for s in range(1,self.ROW):
                permuts=self.hsmat[-1,:]
                if s in permuts:
                    S+=[self.A[-1,int(list(permuts).index(s))]*value]
                else:
                    S+=[0.0]
 
            return(value,H,S)

        else:
            return(value)

    def find_value(self,strategy):
        continuer=True
        while continuer:
            col,row=self.find_pivot()
            if row==-100:
                return(self.printer(strategy))
                break
            self.A=self.pivot(col,row)
    
    
def value_matrix_pivot(A,strategy):
    piv=Pivot_method(A)
    B=piv.find_value(strategy)  
    return(B)




def value_matrix_vertex(A):
    game=nash.Game(A)
    columnP=[] 
    for eq in game.vertex_enumeration(): #game.support_enumeration() ok: #Tous les équilibres
        columnP=eq[1] 
    H_prob=np.around(columnP, decimals=3)
    if list(columnP):
        value=max(np.dot(columnP,A[k,:]) for k in range(A.shape[0]))
    else:
        #print PURE
        print("no eq")
        value=A[0,0] #weird case 
    return(value,H_prob)


def value_vertex(A):
    game=nash.Game(A)
    columnP=[] #default value
    columnS=[]
    for eq in game.vertex_enumeration(): #game.support_enumeration() ok: #Tous les équilibres
        columnP=eq[1] 
        columnS=eq[0] 
    H_prob=np.around(columnP, decimals=12)
    S_prob=np.around(columnS, decimals=12)
    value=A[0,0]
    if list(columnP):
        value=max(np.dot(columnP,A[k,:]) for k in range(A.shape[0]))
    else:
        value=A[0,0]
    return(value,H_prob,S_prob)




class Gamma_k():
    def __init__(self, n,k,p,algo): #n=i cachettes, k=total looks
        self.n=n
        self.k=k
        self.p=p #Vecteur proba
        self.matrixk=np.asarray([[0.0 for i in range(n)] for j in range(math.factorial(n))])#Matrix de float sinon arrondi à 0...
        self.matrixid=np.asarray([[0.0 for i in range(n)] for j in range(math.factorial(n))])#Matrix de float sinon arrondi à 0...
        self.permut=list(itertools.permutations(list(range(1, self.n+1))))
        self.signmatrixk=np.asarray([[-1.0 for i in range(n)] for j in range(math.factorial(n))])
        self.tractablematrix=[]   
        self.algo=algo
    def column_index(self,val_list,printer):
        for idx,perm in enumerate(self.permut): #n! Rows 0,1,2,3,4,5
            for cachette in range(self.n): # n Column 0,1,2
                self.matrixid[idx][cachette]=list(perm).index(cachette+1)+1 #décalage 0 et 1
                #print(self.matrixk)
                if self.matrixid[idx][cachette]>self.k:
                    self.signmatrixk[idx][cachette]=0   
                else:
                    P=self.p[cachette] #str vaut k-1
                    value=max(0,int(self.k-self.matrixid[idx][cachette]))
                    self.signmatrixk[idx][cachette]=P +(1-P)*val_list[value] #val_list[value] #-1 #p[idx]+(1-p[idx])*self.dicovalues["0"]
        if printer:
            print(self.signmatrixk)
            
        if self.algo==0:
            V,H_prob,S_prob=value_vertex(self.signmatrixk)
        else:
            V,H_prob,S_prob=value_matrix_pivot(self.signmatrixk,True)
        return V,H_prob,S_prob 


def plots_only(probas,ax1, xlims, quoi,algo):
    maxiter=xlims[1] #rajouter 1 pour avoir nombre itérations
    values=[0]
    Hprobas=np.zeros((len(probas),maxiter))
    Sprobas=np.zeros((math.factorial((len(probas))),maxiter))
    for k in range(1,maxiter):
        G=Gamma_k(len(probas),k,probas,algo)
        V,H_prob,S_prob =G.column_index(values,False)
        values+=[V]
        Hprobas[:,k]=H_prob
        Sprobas[:,k]=S_prob
    Xaxis=np.linspace(1,maxiter,maxiter)    
    lsbis=['-','--','-.',':']
    if quoi==0:
        for k in range(len(probas)):
            ax1.plot(Xaxis[1:maxiter]-1,Hprobas[k,1:],label='h'+str(k+1),linestyle='-')
        ax1.set_xlim(xlims)
        ax1.set_ylim([0,1.01])
    else:
        for k in range(math.factorial(len(probas))):
            ax1.plot(Xaxis[1:maxiter]-1,Sprobas[k,1:],label='s'+str(k+1),linestyle=lsbis[int(k%len(probas))])
        ax1.set_xlim(xlims)
        ax1.set_ylim([0,1.01])
        
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    return(values,Hprobas,Sprobas)


def returnMatrix(probas,index, quoi,algo):
    maxiter=9 #rajouter 1 pour avoir nombre itérations
    values=[0]

    for k in range(1,maxiter):
        G=Gamma_k(len(probas),k,probas,algo)
        if k==index:
            V,_,_ =G.column_index(values,True)
        else:
            V,_,_ =G.column_index(values,False)
        values+=[V]

# Example of payoff matrix with parameters

In [6]:
p1, p2, p3, f1, f2, f3, v1, v2, v3 = symbols('p1 p2 p3 f1 f2 f3 v1 v2 v3')

A=np.array([[p1+(1-p1)*v3,p2+(1-p2)*v2,p3+(1-p3)*v1],
            [p1+(1-p1)*v3,p2+(1-p2)*v1,p3+(1-p3)*v2],
            [p1+(1-p1)*v2,p2+(1-p2)*v3,p3+(1-p3)*v1],
            [p1+(1-p1)*v1,p2+(1-p2)*v3,p3+(1-p3)*v2],
            [p1+(1-p1)*v2,p2+(1-p2)*v1,p3+(1-p3)*v3],
            [p1+(1-p1)*v1,p2+(1-p2)*v2,p3+(1-p3)*v3]])
A=np.insert(A,3,np.array([1,1,1,1,1,1]),axis=1)
A=np.insert(A,6,np.array([-1,-1,-1,0]),axis=0)

simplify(A)


[[p1 - v3*(p1 - 1), p2 - v2*(p2 - 1), p3 - v1*(p3 - 1), 1], [p1 - v3*(p1 - 1), p2 - v1*(p2 - 1), p3 - v2*(p3 - 1), 1], [p1 - v2*(p1 - 1), p2 - v3*(p2 - 1), p3 - v1*(p3 - 1), 1], [p1 - v1*(p1 - 1), p2 - v3*(p2 - 1), p3 - v2*(p3 - 1), 1], [p1 - v2*(p1 - 1), p2 - v1*(p2 - 1), p3 - v3*(p3 - 1), 1], [p1 - v1*(p1 - 1), p2 - v2*(p2 - 1), p3 - v3*(p3 - 1), 1], [-1, -1, -1, 0]]

In [7]:
A2=pivot(0,0,A)

In [8]:
simplify(A2[-1,1]>0)

(-p1 + p2 - v2*(p2 - 1) + v3*(p1 - 1))/(p1 - v3*(p1 - 1)) > 0

In [9]:
A3=pivot(2,1,A2)

In [10]:
simplify(A3[-1,2]>0)

(-p1*p2*v2 - p1*p2*v3 + 2*p1*p2 + p1*p3*v1 - p1*p3 - p1*v1 + p1*v2 + p1*v3 - p1 + p2*p3*v1 - p2*p3 - p2*v1 + p2*v2 + p2*v3 - p2 - 2*p3*v1 + 2*p3 + 2*v1 - v2 - v3)/(p1*p2*v2 + p1*p2*v3 - 2*p1*p2 - p1*v2 - p1*v3 + p1 - p2*v2 - p2*v3 + p2 + v2 + v3) > 0

In [11]:
A4=pivot(1,2,A3)